# MNIST TEST
MNIST data로 학습시킨 모델로 직접 쓴 손글씨 test 후 정확도 판단

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Model

In [2]:
# CNN 아키텍처 구성을 위한 클래스 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1)

        # Fully Connected Layer
        self.fc = nn.Linear(32*7*7, 10)
        

    def forward(self, x):
        # Convolutional Layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #flatten
        x = x.view(x.size(0),-1)
        print(x.shape)
        x = self.fc(x)
        

        
        return x

## Training

## Test Custom Dataset

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import glob
import os
from PIL import Image
import numpy as np

import torch.optim as optim

In [12]:
class CustomDataset(Dataset):
    def __init__(self,img_path, transform=None):
        self.transfrom = transform
        file_list =['*.jpg','*.png']
        for files in file_list:
            self.image_paths = glob.glob(os.path.join(img_path,files))
        
    def __getitem__(self, index):
        #이미지 파일 라벨링
        image_path = self.image_paths[index]
        file_name = image_path.split('\\')[-1]
        label = file_name[-5]
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        convert_tensor = transforms.ToTensor()  

        # try:
        #gray scale
        image = Image.open(image_path).convert('L')
        #resize
        image = image.resize((28,28))
        #To tensor
        img = convert_tensor(image)
        img = img.unsqueeze(dim=0)
        
        return {
        "img": torch.tensor(img, dtype=torch.float),
        "label": torch.tensor(label, dtype=torch.float)
    }
        
        # except Exception:
        #     print("읽을 수 없는 이미지 파일 입니다")
        #     return



    def __len__(self):
        return len(self.image_paths)



In [13]:
if __name__ == '__main__':

    #이미지 경로
    img_path = './MNIST_data/'
    dataset = CustomDataset(img_path)
    #Custondata 생성
    for i in dataset:
        pass


C:\Users\iiile\AppData\Local\Temp\ipykernel_34060\787438817.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "img": torch.tensor(img, dtype=torch.float),


TypeError: new(): invalid data type 'str'

In [5]:
if __name__ == '__main__':

    #이미지 경로
    img_path = './MNIST_data/'
    #Custondata 생성
    dataset = CustomDataset(img_path)

    #Define model, loss, optim
    model = CNN()
    criterion = nn.CrossEntropyLoss()   # 다중 클래스 분류 문제
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):

        for label, image in dataset:
            optimizer.zero_grad()

            # input_image = image.upsqueeze(dim=0)
            outputs = model(image)

            loss = criterion(outputs, label)

            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")




TypeError: new(): invalid data type 'str'

### resize

### labeling

## Test